In [1]:
import pandas as pd
from sqlite3 import connect

sumos = pd.read_csv("../data/cleaned.csv")
sumos.head()

,basho,id,rikishi,rank,height,weight,bmi,heya,shusshin,outcome,kimarite,opponent id,opponent
0,1983.01,4140,Chikubayama,J13w,175.5,115.0,37.3,Miyagino,Fukuoka,0,yorikiri,4306,Ofuji
1,1983.01,4306,Ofuji,Ms1e,193.5,145.0,38.7,Kokonoe,Tokyo,1,yorikiri,4140,Chikubayama
2,1983.01,1337,Tochitsukasa,J12w,178.5,130.0,40.8,Kasugano,Aichi,1,oshidashi,4323,Shiraiwa
3,1983.01,4323,Shiraiwa,J13e,182.4,103.0,31.0,Isegahama,Akita,0,oshidashi,1337,Tochitsukasa
4,1983.01,4097,Tamakiyama,J12e,182.0,134.0,40.5,Kataonami,Fukuoka,0,yorikiri,4319,Harunafuji


In [3]:
rikishi_list = []
for i in sumos["rikishi"].unique():
    rikishi_list.append(i)

In [7]:
#What is the highest rank each rikishi on the list achieved?
#The information in rikishi1_rank includes a designation of "e" or "w" ("east" or "west" on the banzuke), but as that is immaterial to ranking, we can strip it out.
#A simple excision of the final character doesn't work, so I can't do the following: sumo.rikishi1_rank.str[:-1]
#See, some rankings appear on the spreadsheet with an "HD" at the end, so I have to work with a list of characters to remove in order to clean this out.

char_to_remove = ['e', 'w', 'HD']
cleaned_rankings = sumos['rank']
for character in char_to_remove:
    cleaned_rankings = cleaned_rankings.str.replace(character, '')
#print(cleaned_rankings)


#But this ranking only shows a rikishi's rank for each match. That rank stays the same throughout any given tournament, but can be adjusted for the next tournament.
#So we want to see what the highest rank achieved by each rikishi.
#Starting at the top, the ranks are Y (Yokozuna), O (Ozeki), S (Sekiwake), K (Komusubi). These are the san'yaku ranks, the highest set of rankings.
#Then comes M (Maegashira, the best being M1 then increasing in number), then J (Juryo, J1, then increasing), Ms (Makushita, Ms1, then increasing).
#For the top san'yaku ranks, we see things like Y1 and Y2 on the banzuke, but that is just for matching purposes. The numberings only matter at lower levels,
#which is why the following value assignment gives equal values to each Y/O/S/K rank but differing values for the lower ranks.
ranking_dict = {'Y1': '1', 'Y2': '1', 'Y1YO': '1', 'Y2YO': '1', 'O1': '2', 'O2': '2', 'O3': '2', 'S1': '3', 'S2': '3', 'K1': '4', 'K2': '4', 'K3': '4', 'K4': '4', 'M1': '5', 'M2': '6', 'M3': '7', 'M4': '8', 'M5': '9', 'M6': '10', 'M7': '11',
                'M8': '12', 'M9': '13', 'M10': '14', 'M11': '15', 'M12': '16', 'M13': '17', 'M14': '18', 'M15': '19', 'M16': '20', 'M17': '21', 'M18': '22', 'J1': '23', 'J2': '24', 'J3': '25',
                'J4': '26', 'J5': '27', 'J6': '28', 'J7': '29', 'J8': '30', 'J9': '31', 'J10': '32', 'J11': '33', 'J12': '34', 'J13': '35', 'J14': '36', 'Ms1': '37', 'Ms2': '38', 'Ms3': '39',
                'Ms4': '40', 'Ms5': '41', 'Ms6': '42', 'Ms7': '43', 'Ms8': '44', 'Ms9': '45', 'Ms10': '46', 'Ms11': '47', 'Ms12': '48', 'Ms13': '49', 'Ms14': '50'}

#The following is a test to make sure no rankings are excluded from this cleanup set. If test = [] after this, we're good to go.
test = []
for i in sumos["rank"].unique():
    if (i in ranking_dict):
        test
    else:
        test.append(i)

test

['J13w',
 'Ms1e',
 'J12w',
 'J13e',
 'J12e',
 'J11w',
 'J10w',
 'J11e',
 'J10e',
 'J9w',
 'J9e',
 'J8w',
 'J8e',
 'J7w',
 'J6w',
 'J7e',
 'J6e',
 'J5w',
 'J4w',
 'J5e',
 'J3w',
 'J4e',
 'J2w',
 'J3e',
 'J2e',
 'J1w',
 'J1e',
 'M14w',
 'M14e',
 'M13w',
 'M13e',
 'M12w',
 'M11w',
 'M12e',
 'M11e',
 'M10w',
 'M9w',
 'M10e',
 'M9e',
 'M8w',
 'M8e',
 'M7w',
 'M7e',
 'M6w',
 'M6e',
 'M4w',
 'M3e',
 'M4e',
 'M2e',
 'M2w',
 'M1w',
 'S1w',
 'M1e',
 'O2wHD',
 'S1e',
 'O1w',
 'M5w',
 'O1e',
 'Y2eHD',
 'K1e',
 'M5e',
 'Y1w',
 'Y1e',
 'K1w',
 'Ms1w',
 'Ms2e',
 'Ms3e',
 'Ms3wHD',
 'Ms3w',
 'Ms4w',
 'Ms2w',
 'M3w',
 'O2eHD',
 'Ms4e',
 'Ms5e',
 'Ms6w',
 'Ms5w',
 'Ms7w',
 'Ms6e',
 'Ms8e',
 'Ms10w',
 'Ms7e',
 'M9eHD',
 'O3eHD',
 'Ms14e',
 'Ms9w',
 'Ms8w',
 'S2eHD',
 'Ms9e',
 'O3wHD',
 'Y2wHD',
 'M15w',
 'M15e',
 'M16e',
 'K2eHD',
 'M16w',
 'K2wHD',
 'S2wHD',
 'O2w',
 'O2e',
 'K2w',
 'Y1eHD',
 'S2w',
 'S2e',
 'Y2e',
 'K2e',
 'Y2w',
 'O3w',
 'O3e',
 'J14w',
 'J14e',
 'M17e',
 'M17w',
 'Ms10e',
 'M18e',
 '

In [4]:
# There are two lines for each match, one for the winner and one for the loser.
# So as not to show duplicates of every match, we'll just show the winners with sumo_only_wins
sumo_only_wins = sumos[
    sumos.outcome == int("1")
]  # could also use sumo.loc[sumo.rikishi1_win == 1]

sumo_only_wins[["rikishi", "kimarite"]]
# sumo_only_wins

,rikishi,kimarite
1,Ofuji,yorikiri
2,Tochitsukasa,oshidashi
5,Harunafuji,yorikiri
7,Hakuryuyama,tsukidashi
9,Hachiya,hatakikomi
...,...,...
227934,Kotonowaka,yorikiri
227936,Asanoyama,yorikiri
227937,Takanosho,hikiotoshi
227939,Hoshoryu,uwatenage
